### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2022
refund = 112000.00   # 112,167.31
dividend = 544_000
expense  = 608_000

today = date.today()
today

datetime.date(2022, 11, 26)

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sells.date DESC, name


In [3]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,BH,2022,11,2022,11,220.00,210.00,10.00,500,"110,000.00","105,000.00","5,000.00",4.76,"4,523.80",DTD
1,DOHOME,2022,11,2021,8,14.20,20.25,-6.05,"2,400","34,080.00","48,600.00","-14,520.00",-29.88,"-14,703.13",DTD
2,DOHOME,2022,11,2022,5,13.70,20.25,-6.55,"2,400","32,880.00","48,600.00","-15,720.00",-32.35,"-15,900.46",DTD
3,VNG,2022,11,2022,8,6.15,6.70,-0.55,"15,000","92,250.00","100,500.00","-8,250.00",-8.21,"-8,676.93",DTD
4,SCC,2022,11,2022,1,340.00,384.00,-44.00,300,"102,000.00","115,200.00","-13,200.00",-11.46,"-13,681.07",DTD
5,SYNEX,2022,11,2022,3,16.10,24.50,-8.40,"3,000","48,300.00","73,500.00","-25,200.00",-34.29,"-25,469.78",DTD
6,KCE,2022,11,2022,3,44.25,57.50,-13.25,"1,000","44,250.00","57,500.00","-13,250.00",-23.04,"-13,475.37",DTD
7,STA,2022,11,2022,3,19.20,25.75,-6.55,"2,500","48,000.00","64,375.00","-16,375.00",-25.44,"-16,623.91",HD
8,PTTGC,2022,10,2022,3,43.50,51.25,-7.75,"3,000","130,500.00","153,750.00","-23,250.00",-15.12,"-23,879.59",DTD
9,BCH,2022,10,2021,12,18.40,20.50,-2.10,"3,000","55,200.00","61,500.00","-6,300.00",-10.24,"-6,558.48",DTD


In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name  
2022       1           2019      11         MCS      -10,846.64
                       2021      3          IVL        6,925.23
                                 6          IVL       24,327.22
                                 7          IVL       18,150.32
                                            SAT        7,998.06
                                                        ...    
           11          2022      3          STA      -16,623.91
                                            SYNEX    -25,469.78
                                 5          DOHOME   -15,900.46
                                 8          VNG       -8,676.93
                                 11         BH         4,523.80
Name: net, Length: 87, dtype: float64

In [5]:
sells_df.groupby(['name'])[['gross','net']].sum().sample(5)

,gross,net
name,,
TISCO,"-11,000.00","-12,707.70"
SCC,"-13,200.00","-13,681.07"
TOP,"32,500.00","29,557.48"
DCC,"4,000.00","3,477.28"
SPALI,"-4,500.00","-5,367.13"


In [6]:
total_df = sells_df.groupby(['name'])[['gross','net']].sum()
total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

,gross,net
name,,
IVL,"82,350.00","77,529.62"
IMH,"49,300.00","47,826.43"
SINGER,"37,875.00","36,948.89"
TOP,"32,500.00","29,557.48"
GLOBAL,"25,800.00","23,720.19"


In [7]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(box_file)

In [8]:
sells_df.groupby(['sell_month']).net.sum()

sell_month
1     117,432.48
2     -81,958.99
3      35,536.86
4         797.09
5     -44,273.12
6      -9,641.76
7     -17,849.25
8       3,613.24
9       8,317.61
10    -37,156.96
11   -104,006.85
Name: net, dtype: float64

In [9]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(-129189.65000000001, 544000, 414810.35)

In [10]:
profit = sells_df.net.sum()
net = profit + dividend - expense
round(profit,0),dividend,expense, round(net,0)

(-129190.0, 544000, 608000, -193190.0)

### Bottom line of up to date transactions

In [11]:
total = refund + net
total

-81189.65000000002